In [1]:
import numpy as np

In [2]:
DIGITS = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"
base = 16

In [3]:
def decimal_to_base(number : int, base : int, length : int):
    """
    Converts decimal number to another base
    Args:
        number(int): decimal number to convert into another base
        base(int): target base to convert the number into
        length(int): length of the representation
    
    Returns:
        str: number in the new base
    
    Note:
    
    """
    if (not isinstance(base, int)) or (not isinstance(length, int)) or (base <= 1):
        return "invalid input"
    maxpow = base ** (length-1)
    if number >= maxpow or number < -maxpow:
        return "number out of bounds"
    
    
    answer = ["" for _ in range(length)]
    for i in range(length):
        
        if i == 0:

            if number < 0:
                answer[i] = DIGITS[1]
                number += maxpow
            else:
                answer[i] = DIGITS[0]
        else:
            answer[i] = DIGITS[int(number//maxpow)]
            number -= (number//maxpow)*maxpow
                        
        maxpow /= base

        
    return "".join(answer)

In [4]:
def base_to_decimal(string : str , base : int , signed : bool = True):
    """
    Converts integer in certain base to decimal form.
    Args:
        string(str): string of number in given base
        base(int): given base
        signed(bool): True if given string represents signed integer, else false
    
    Returns:
        int: decimal form of the string converted from the given base.
    
    Note:
    
    """
    if (not isinstance(base, int))  or (base <= 1):
        return "invalid input"
    maxpow = base ** (len(string)-1)

    answer = 0
    for i in range(len(string)):
        if i == 0:
            answer += (-2*signed + 1) * maxpow * DIGITS.index(string[i])
        else:
            answer += maxpow * DIGITS.index(string[i])
        maxpow /= base
    return answer


In [5]:
decimal_to_base(100, 16, 6)

'000064'

In [6]:
string = "16"
base = 16

print(  (-1*int(string[0]))* (base ** (len(string)-1)) + int(string[1:], 16) )

-10


In [7]:
def multiply(A:str, B:str, base:int):
    """
    Multiplies two integers represented as strings given in a certain base.

    Args:
        A(str): The first string in the given base.
        B(str): The second string in the given base.
        base(int): Base of the number system.
    Returns:
        str: Product of the strings by our algorithm.
    Note:
        This is just an intermediate function made while writing the dot_product() function. 
        I have constrained A and B to be of same length but if you want, please feel free 
        to bang your head to accomodate other cases which are irrelevant for our use case.
    """
    if (not isinstance(base, int))  or (base <= 1) or len(A) != len(B):
        return "invalid input"
    L = len(A)
    answer = [0 for _ in range(2*L)]

    arr1, arr2 = [DIGITS.index(A[L-1-(L-1)])] ,[ DIGITS.index(B[L-1-(L-1)])]

    answer[2*L - 1 - (2*(L - 1))] = np.dot(arr1, arr2)


    for i in range(L-1):
        arr1, arr2 = [DIGITS.index(A[L-1-(L-1)]),  DIGITS.index(A[L-1-(i)]) ], [DIGITS.index(B[L-1-(i)]),  DIGITS.index(B[L-1-(L-1)]) ]
        answer[2*L-1 - (L + i - 1)] -= np.dot(arr1, arr2)

    for a in range(2*(L-2)+1):
        if a < L-1:
            arr1, arr2 = [  DIGITS.index(A[L-1-i])   for i in range(a+1)  ], [  DIGITS.index(B[L-1-(a-i)]) for i in range(a+1)  ]
            answer[2*L-1-a] += np.dot(arr1, arr2)

        else:
            arr1, arr2 = [  DIGITS.index(A[L-1-i])  for i in range(a-(L-2), L-1)  ], [   DIGITS.index(B[L-1-(a-i)]) for i in range(a-(L-2), L-1)  ]
            answer[2*L-1-a] += np.dot(arr1, arr2)

    carry = 0
    for i in range(2*L-1, -1, -1):
        if i == 0:
            if carry == -1:
                answer[i] = 1
            elif carry == 0:
                answer[i] = 0
            else:
                print("exception occured ", A, B)
        else:
            answer[i] += carry
            carry = answer[i]//base
            answer[i] = answer[i]%base

    answer = [DIGITS[i] for i in answer]

    return "".join(answer)

print( multiply("16", "01", base))

1FF6


In [8]:
def dot_product(A:list, B:list, base:int):
    """
    Dot Product of two lists of strings in a base.
    
    Args:
        A(list) : The first list where each element is an integer in a base
        B(list) : The second list where each element is an integer in a base
        base(int) : Base of the number system
    Returns:
        string: Dot product of the numbers of base.
    
    Note:
        lengths of all strings in A and B are same.
    
    """
    if (not isinstance(base, int))  or (base <= 1) or len(A) != len(B):
        return "invalid input"
    

    def flatten(lis):
        """
        Flattens a 2-d list.

        Args:
            lis(list) : The list to flatten

        Returns:
            list : flattened list
        """
        return  [item for sublist in lis for item in sublist]

    # Length of each string
    L = len(A[0])
    #List to store final dot product
    answer = [0 for _ in range(2*L)]
    
    # Temporary arrays used all over. Each element of these lists is a length = 1 number in the given base
    # Everywhere we used arr1 and arr2, they are the arrays that will be the only arrays actually placed on chip "to perform logic".
    # There will be a crossbar array to store the pre-processed input values
    arr1, arr2 = [[DIGITS.index(A[n][L-1-(L-1)])] for n in range(len(A))] ,[[ DIGITS.index(B[n][L-1-(L-1)])] for n in range(len(A))]
    
    # First summation part of the algorithm
    #Setting the 1st index of answer according to the algorithm
    answer[2*L - 1 - (2*(L - 1))] = np.dot(flatten(arr1), flatten(arr2))

    # Second part of the algorithm. The minus part
    for i in range(L-1):
        arr1, arr2 = [[DIGITS.index(A[n][L-1-(L-1)]),  DIGITS.index(A[n][L-1-(i)]) ] for n in range(len(A))], [[DIGITS.index(B[n][L-1-(i)]),  DIGITS.index(B[n][L-1-(L-1)]) ] for n in range(len(A))]
        # Minus operation can be achieved by pulling the appropriate value of current using current mirrors, or using a differential amplifier etc....
        # the exact implementation can be figured out but that is not much of a concern because it is pretty standard problem and there are standard solutions.
        answer[2*L-1 - (L + i - 1)] -= np.dot(flatten(arr1), flatten(arr2))
    
    # Final part of the algorithm, the double summation which we then reduced to a single summation.
    for a in range(2*(L-2)+1):
        if a < L-1:
            arr1, arr2 = [[  DIGITS.index(A[n][L-1-i])   for i in range(a+1)  ] for n in range(len(A))], [[  DIGITS.index(B[n][L-1-(a-i)]) for i in range(a+1)  ] for n in range(len(A))]

            answer[2*L-1-a] += np.dot(flatten(arr1), flatten(arr2))

        else:
            arr1, arr2 = [[  DIGITS.index(A[n][L-1-i])  for i in range(a-(L-2), L-1)  ] for n in range(len(A))], [[   DIGITS.index(B[n][L-1-(a-i)]) for i in range(a-(L-2), L-1)  ] for n in range(len(A))]
            answer[2*L-1-a] += np.dot(flatten(arr1), flatten(arr2))

    # During multipying big numbers, we multiply small numbers and then send carry forward. This is that part of the algorithm.
    #This part will be taken care by software. This is an O(L^2) problem, so it is not much of a burden on the overall time complexity.   
    carry = 0
    for i in range(2*L-1, -1, -1):
        if i == 0:
            if carry<=0:
                answer[i] = -carry
            else:
                # For some reason if my algorithm breaks.....just in case
                print("exception occured", carry, A, B)
        else:
            answer[i] += carry
            carry = answer[i]//base
            answer[i] = answer[i]%base

    # Finally joining the answer to return
    answer = [DIGITS[i] for i in answer]
    return "".join(answer)
    

num1, num2 = [-64732, 64732], [36191, 36191]
length = 5
arr1, arr2 = [decimal_to_base(n, base, length) for n in num1], [decimal_to_base(n, base, length) for n in num2]
dot_product(arr1, arr2, base), decimal_to_base((np.dot(num1, num2)), base, 2*length), np.dot(num1, num2)


('0000000000', '0000000000', 0)

Testbench to test multiply function

In [9]:
def test():
    length, base = 6, 16
    num1, num2 = np.random.randint(-base**(length-1), base**(length-1)), np.random.randint(-base**(length-1), base**(length-1))
    if  num1*num2 != base_to_decimal(multiply(  decimal_to_base(int(num1), base, length), decimal_to_base(int(num2), base, length)  ,base), base, True):
        print(num1, num2, "Violated")

for _ in range(100000):
    test()

Testbench to test the final dot product

In [10]:
def test_dot_product():
    length, base = 5, 8
    lower_limit, upper_limit = -base**(length-1), base**(length-1)
    num1, num2 = np.random.randint(lower_limit, upper_limit, size = 15), np.random.randint(lower_limit, upper_limit, size = 15)
    arr1, arr2 = [decimal_to_base(n, base, length) for n in num1], [decimal_to_base(n, base, length) for n in num2]

    
    if (base_to_decimal(dot_product(arr1, arr2, base), base, True) !=  np.dot(num1, num2) ):
        print(num1, num2)


for _ in range(100000):
    test_dot_product()


In [11]:
np.dot(-64732,36191.0)

-2342715812.0

In [12]:
import struct

def double_to_binary_string(d):
    double_as_bytes = struct.pack('d', d)
    double_as_int = struct.unpack('Q', double_as_bytes)[0]
    return format(double_as_int, '064b')

double_to_binary_string(93.6789)

'0100000001010111011010110111001100011000111111000101000001001000'

In [13]:
def binary_to_decimal(s):
    sign, exponent, mantissa = int(s[0]), s[1:12], s[12:]
    exponent, mantissa = base_to_decimal(exponent, 2, False), base_to_decimal(mantissa, 2, False)
    return  (-1)**sign * (1 + mantissa/2**52) *(2**(exponent-1023))
binary_to_decimal(double_to_binary_string(0.9876987))

0.9876987

In [14]:
def preprocessing(num, base:int):
    """
    Converts any float into integer of given base, multiplied by some comstant power of two.
    """
    if (not isinstance(base, int))  or (base <= 1):
        return "invalid input"
    bin_str = double_to_binary_string(num)
    sign, exponent, mantissa = int(bin_str[0]), bin_str[1:12], bin_str[12:]

    print(mantissa)
    exponent = int(base_to_decimal(exponent, 2, False))
    diff = 1023 - exponent 

    if diff >= 0 :
        mantissa = ("0"*diff + mantissa)[:len(mantissa)]
    elif diff < 0 :
        mantissa = mantissa + "0"*(-diff)
    print(mantissa, diff)
    print(len(mantissa))




preprocessing(3, 16)





1000000000000000000000000000000000000000000000000000
10000000000000000000000000000000000000000000000000000 -1
53
